In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from datetime import datetime
from datetime import timedelta, date
import matplotlib.dates as mdates
import math
import warnings

from sklearn import neighbors
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings('ignore')

#from xgboost import XGBClassifier
#from sklearn.metrics import accuracy_score

In [2]:
properties = pd.read_csv('trainTree.csv')
testing = pd.read_csv('testTree.csv')

In [33]:
#prueba
train = properties[10000:]
train = train.drop('id', 1)
train = train.drop('description', 1)
train = train.drop('Unnamed: 0', 1)
train = train.drop('created_on', 1)
train = train.drop('lat', 1)
train = train.drop('lon', 1)

In [34]:
test = properties[0:10000]
t = test.drop('id', 1)
t=  t.drop('description', 1)
t = t.drop('Unnamed: 0', 1)
#cross2= cross2.drop('price_aprox_usd', 1)
t= t.drop('created_on', 1)
t= t.drop('lat', 1)
t= t.drop('lon', 1)
crossValidation = pd.DataFrame()
#crossValidation['price_aprox_usd'] = tes['price_aprox_usd']
#test.info()

In [35]:
#obtengo trainvacios y testvacios con los entries de los respectivos datasets que cumplen:
#en el train set, menos de 10 entradas por place_name_encoded
#en el test set, no tienen entradas en el train set con el mismo place_name_encoded


l = len(properties['place_name_encoded'].value_counts())

#CAMBIO: hardcodeo el valor original (place name encoded va hasta 678)


#uso este porque sé que es bajo
trainvacios = train[train['place_name_encoded']==1]
testvacios = t[t['place_name_encoded']==1]

for x in range(0,l+2):
    if ((len(t[t['place_name_encoded']==x])==0) or (len(train[train['place_name_encoded']==x])<50)) and (x!=1):
    #excluyo el hardcodeado
    
        train0 = train[train['place_name_encoded']==x]
        trainvacios = pd.concat([trainvacios, train0])
        
        test0 = t[t['place_name_encoded']==x]
        testvacios = pd.concat([testvacios, test0])

In [36]:
#los datos restantes tienen más de 10 entradas en train y al menos alguna en test por barrio
#los junto en el dataframe 'trainaptos'

trainaptos= train[~train['place_name_encoded'].isin(trainvacios['place_name_encoded'])]
testaptos= t[~t['place_name_encoded'].isin(testvacios['place_name_encoded'])]
#trainaptos.info()

In [37]:
#Reviso que quedo bien el split

total = pd.concat([trainaptos, trainvacios])
print('Train:')
print('Registros:      ', len(total), ' = ', len(train))
print('Aptos + Vacios: ', len(trainaptos), '+', len(trainvacios))

print('Place_names:    ', len(train['place_name_encoded'].value_counts()), ' = ', len(total['place_name_encoded'].value_counts()))
print('Vacios + Aptos: ', len(trainvacios['place_name_encoded'].value_counts()), '+', len(trainaptos['place_name_encoded'].value_counts()))
print()


total = pd.concat([testaptos, testvacios])
print('Test:')
print('Registros:      ', len(total), ' = ', len(t))
print('Aptos + Vacios: ', len(testaptos), '+', len(testvacios))

print('Place_names:    ', len(t['place_name_encoded'].value_counts()), ' = ', len(total['place_name_encoded'].value_counts()))
print('Vacios + Aptos: ', len(testvacios['place_name_encoded'].value_counts()), '+', len(testaptos['place_name_encoded'].value_counts()))

Train:
Registros:       599670  =  599670
Aptos + Vacios:  575612 + 24058
Place_names:     899  =  899
Vacios + Aptos:  604 + 295

Test:
Registros:       10000  =  10000
Aptos + Vacios:  9897 + 103
Place_names:     361  =  361
Vacios + Aptos:  66 + 295


In [38]:
trainaptos['place_name_encoded'].value_counts()

615    23873
189    22103
784    18079
598    17908
218    14122
637    13372
675    12822
834    12746
145    11814
606    11422
874    11179
16     11129
484    10878
381    10771
446     9939
735     9386
587     8175
42      7451
658     7278
667     7032
601     6672
671     6663
746     6636
722     6539
241     6310
732     6096
815     5953
37      5481
242     5386
46      5039
       ...  
51       101
613       99
562       99
703       93
471       93
337       93
492       91
264       89
330       83
827       79
238       76
236       73
826       69
699       69
271       68
105       66
127       65
614       65
286       63
866       62
327       61
14        60
700       60
416       60
209       59
424       59
321       57
390       52
831       51
267       50
Name: place_name_encoded, Length: 295, dtype: int64

In [42]:
#armo una funcion que devuelva un dataframe con las predicciones del test y train dados y su id

def RandomForest(test,train):
    train = train.drop('place_name_encoded',1)
    
    trainLX = train.drop('price_aprox_usd', 1)
    trainLY = train[['price_aprox_usd']]
    
    #t = test.drop('id',1)
    t = test.drop('price_aprox_usd',1)
    t = t.drop('place_name_encoded',1)
    
    regr = RandomForestRegressor(min_samples_leaf=1, min_samples_split=2,n_estimators=375, n_jobs = -1,max_features='log2')
    predictions = regr.fit(trainLX, trainLY).predict(t)
    predictions = predictions.flatten()

    ids = test.as_matrix()[:,0].flatten()
    results = pd.DataFrame()
    results['id'] = pd.Series(ids.astype('int32'))
    results['price_usd'] = pd.Series(predictions)
    
    
    return results

#armo el dataframe resultados. Empieza con los test y train 'vacios'
#el resto se hará individualmente por place_name uno por uno posteriormente
resultados = RandomForest(testvacios,trainvacios)
resultados.info()

crossValidation = pd.DataFrame()
crossValidation['price_aprox_usd'] = testvacios['price_aprox_usd']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 2 columns):
id           103 non-null int32
price_usd    103 non-null float64
dtypes: float64(1), int32(1)
memory usage: 1.3 KB


In [44]:
#ahora saco las predicciones para los test y train aptos uno por uno
l = len(properties['place_name_encoded'].value_counts())

for x in range(0,l+2):
    if len(testaptos[testaptos['place_name_encoded']==x])!=0:
        testx = testaptos[testaptos['place_name_encoded']==x]
        trainx = trainaptos[trainaptos['place_name_encoded']==x]
        resx = RandomForest(testx,trainx)
        
        crossX = pd.DataFrame()
        crossX['price_aprox_usd'] = testx['price_aprox_usd']
        crossValidation = pd.concat([crossValidation, crossX])
        
        resultados = pd.concat([resultados, resx])

In [45]:
crossValidation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 9892 to 8888
Data columns (total 1 columns):
price_aprox_usd    10000 non-null float64
dtypes: float64(1)
memory usage: 156.2 KB


In [46]:
#debe contener la misma cantidad que testing (14166)
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 10
Data columns (total 2 columns):
id           10000 non-null int32
price_usd    10000 non-null float64
dtypes: float64(1), int32(1)
memory usage: 195.3 KB


In [48]:
score = mean_squared_error(crossValidation['price_aprox_usd'].values, resultados['price_usd'].values)
score

2807173824.6856117

In [13]:
#resultados.set_index(['id'], inplace=True)
#resultados.to_csv('predicciones_RF_placename.csv')